In [ ]:
#!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org selenium
#!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
import time
import shutil
import glob


In [2]:
from bs4 import BeautifulSoup
import pandas as pd

# Teste - get url da nova aba

In [28]:
driver = webdriver.Chrome()

url = "http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/qgpb.def"
driver.get(url)

WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//select[@id='L']")))

# Seleciona o elemento do dropdown para Estabelecimento
dropdown_estabelecimento = driver.find_element(By.XPATH, "//select[@id='L']")
for option in dropdown_estabelecimento.find_elements(By.TAG_NAME, 'option'):
    if option.text == 'Estabelecimento':
        option.click()
        print("Opção 'Estabelecimento' selecionada:", option.text)

# Seleciona todas as opções no dropdown de Incremento
dropdown_incremento = driver.find_element(By.XPATH, "//select[@id='I']")
for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
    option.click()
    #print("Opção selecionada no dropdown de Incremento:", option.text)

# Desmarca todas as opções no dropdown de Incremento
for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
    if option.is_selected():
        option.click()
        #print("Opção desmarcada no dropdown de Incremento:", option.text)

# Seleciona todas as opções no dropdown de Incremento novamente
for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
    option.click()
    #print("Opção selecionada no dropdown de Incremento:", option.text)

# Zera as opções do dropdown de Arquivos
dropdown_arquivos = driver.find_element(By.XPATH, "//select[@id='A']")

# Desmarca todas as opções selecionadas
for option in dropdown_arquivos.find_elements(By.TAG_NAME, 'option'):
    if option.is_selected():
        option.click()

# Seleciona todos os meses de 2019 no dropdown de Arquivos
ano = '2019'
for option in dropdown_arquivos.find_elements(By.TAG_NAME, 'option'):
    if ano in option.text:
        option.click()
        print("Opção selecionada no dropdown de Arquivos:", option.text)
        ano_selecionado = option.text

# Marca a opção "Tabela com bordas"
tabela_com_bordas = driver.find_element(By.XPATH, "//input[@value='table']")
tabela_com_bordas.click()
print("Opção 'Tabela com bordas' marcada")

# Clique no botão "Mostra"
botao_mostra = driver.find_element(By.XPATH, "//input[@class='mostra']")
botao_mostra.click()
print("Botão 'Mostra' clicado")



# Espera até que a nova aba seja aberta
WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))

# Alterna para a nova aba
janela_atual = driver.window_handles[1]
driver.switch_to.window(janela_atual)

print("URL após redirecionamento:", driver.current_url)

driver.quit()

Opção 'Estabelecimento' selecionada: Estabelecimento
Opção selecionada no dropdown de Arquivos: Dez/2019
Opção selecionada no dropdown de Arquivos: Nov/2019
Opção selecionada no dropdown de Arquivos: Out/2019
Opção selecionada no dropdown de Arquivos: Set/2019
Opção selecionada no dropdown de Arquivos: Ago/2019
Opção selecionada no dropdown de Arquivos: Jul/2019
Opção selecionada no dropdown de Arquivos: Jun/2019
Opção selecionada no dropdown de Arquivos: Mai/2019
Opção selecionada no dropdown de Arquivos: Abr/2019
Opção selecionada no dropdown de Arquivos: Mar/2019
Opção selecionada no dropdown de Arquivos: Fev/2019
Opção selecionada no dropdown de Arquivos: Jan/2019
Opção 'Tabela com bordas' marcada
Botão 'Mostra' clicado
URL após redirecionamento: http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sih/cnv/qgpb.def


# Função para baixar 1 csv de 1 uf  com todo o período de ano

In [30]:
def get_data(uf):
    # Diretório de download
    download_path = os.path.join(os.getcwd(), "csv_files")

    if not os.path.exists(download_path):
        os.makedirs(download_path)
        print(f"Pasta '{download_path}' criada.")

    # Configura as opções do Chrome para definir o diretório de download
    chrome_options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": download_path}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(options=chrome_options)

    url = f"http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/qg{uf}.def"
    driver.get(url)

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//select[@id='L']")))

    # Seleciona o elemento do dropdown para Estabelecimento
    dropdown_estabelecimento = driver.find_element(By.XPATH, "//select[@id='L']")
    for option in dropdown_estabelecimento.find_elements(By.TAG_NAME, 'option'):
        if option.text == 'Estabelecimento':
            option.click()
            print("Opção 'Estabelecimento' selecionada:", option.text)

    # Seleciona todas as opções no dropdown de Incremento
    dropdown_incremento = driver.find_element(By.XPATH, "//select[@id='I']")
    for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
        option.click()
        #print("Opção selecionada no dropdown de Incremento:", option.text)

    # Desmarca todas as opções no dropdown de Incremento
    for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
        if option.is_selected():
            option.click()
            #print("Opção desmarcada no dropdown de Incremento:", option.text)

    # Seleciona todas as opções no dropdown de Incremento novamente
    for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
        option.click()
        #print("Opção selecionada no dropdown de Incremento:", option.text)

    # opções do dropdown de Arquivos
    dropdown_arquivos = driver.find_element(By.XPATH, "//select[@id='A']")
    for option in dropdown_arquivos.find_elements(By.TAG_NAME, 'option'):
        if option.is_selected():
            option.click()

    # Seleciona todos os anos de 2019 a 2023 
    for year in range(2019, 2024):
        year_str = str(year)
        for option in dropdown_arquivos.find_elements(By.TAG_NAME, 'option'):
            if year_str in option.text:
                option.click()
                print("Opção selecionada no dropdown de Arquivos:", option.text)

    # Marca a opção "Tabela com bordas"
    tabela_com_bordas = driver.find_element(By.XPATH, "//input[@value='table']")
    tabela_com_bordas.click()
    print("Opção 'Tabela com bordas' marcada")

    # Clicar no botão "Mostra"
    botao_mostra = driver.find_element(By.XPATH, "//input[@class='mostra']")
    botao_mostra.click()
    print("Botão 'Mostra' clicado")

    # Espera até que uma nova aba seja aberta
    WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))

    # Alterna para a nova aba
    janela_atual = driver.window_handles[1]
    driver.switch_to.window(janela_atual)

    # Clique no botão "Copia como .CSV"
    botao_csv = driver.find_element(By.XPATH, "//a[contains(text(), 'Copia como .CSV')]")
    botao_csv.click()
    print("Botão 'Copia como .CSV' clicado")

    # Aguarda um tempo para o download ser concluído
    time.sleep(5)

    # Captura o nome do arquivo baixado
    arquivos_csv = glob.glob(os.path.join(download_path, "sih_cnv_*.csv"))  
    arquivo_baixado = arquivos_csv[0] if arquivos_csv else None

    if arquivo_baixado is not None:
        #print(f"Arquivo baixado: {os.path.basename(arquivo_baixado)}")
        #print(f"Localização: {os.path.dirname(arquivo_baixado)}")
        arquivo_novo = os.path.join(download_path, f"{uf}_2019_2023.csv")
        os.rename(arquivo_baixado, arquivo_novo)
        #print(f"Arquivo '{arquivo_baixado}' renomeado para '{os.path.basename(arquivo_novo)}'")
        print(f"'{os.path.basename(arquivo_novo)}' baixado em csv_files.")

    else:
        print("Nenhum arquivo CSV foi encontrado.")

# Chamando a função com a UF desejada
uf_desejada = 'pb'
get_data(uf_desejada)

Opção 'Estabelecimento' selecionada: Estabelecimento
Opção selecionada no dropdown de Arquivos: Dez/2019
Opção selecionada no dropdown de Arquivos: Nov/2019
Opção selecionada no dropdown de Arquivos: Out/2019
Opção selecionada no dropdown de Arquivos: Set/2019
Opção selecionada no dropdown de Arquivos: Ago/2019
Opção selecionada no dropdown de Arquivos: Jul/2019
Opção selecionada no dropdown de Arquivos: Jun/2019
Opção selecionada no dropdown de Arquivos: Mai/2019
Opção selecionada no dropdown de Arquivos: Abr/2019
Opção selecionada no dropdown de Arquivos: Mar/2019
Opção selecionada no dropdown de Arquivos: Fev/2019
Opção selecionada no dropdown de Arquivos: Jan/2019
Opção selecionada no dropdown de Arquivos: Dez/2020
Opção selecionada no dropdown de Arquivos: Nov/2020
Opção selecionada no dropdown de Arquivos: Out/2020
Opção selecionada no dropdown de Arquivos: Set/2020
Opção selecionada no dropdown de Arquivos: Ago/2020
Opção selecionada no dropdown de Arquivos: Jul/2020
Opção selec

# Função - 1 csv de cada uf para cada ano 

In [23]:
def get_data_por_ano(uf, ano):
    # Diretório de download
    download_path = os.path.join(os.getcwd(), "csv_files")

    if not os.path.exists(download_path):
        os.makedirs(download_path)
        print(f"Pasta '{download_path}' criada.")
    else:
        print(f"Baixando CSV da UF {uf} do ano {ano}...")
    try:
        # Configura as opções do Chrome para definir o diretório de download
        chrome_options = webdriver.ChromeOptions()
        prefs = {"download.default_directory": download_path}
        chrome_options.add_experimental_option("prefs", prefs)

        driver = webdriver.Chrome(options=chrome_options)

        url = f"http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/qg{uf}.def"
        driver.get(url)

        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//select[@id='L']")))

        # Seleciona o elemento do dropdown para Estabelecimento
        dropdown_estabelecimento = driver.find_element(By.XPATH, "//select[@id='L']")
        for option in dropdown_estabelecimento.find_elements(By.TAG_NAME, 'option'):
            if option.text == 'Estabelecimento':
                option.click()
                #print("Opção 'Estabelecimento' selecionada:", option.text)

        # Seleciona todas as opções no dropdown de Incremento
        dropdown_incremento = driver.find_element(By.XPATH, "//select[@id='I']")
        for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
            option.click()
            #print("Opção selecionada no dropdown de Incremento:", option.text)

        # Desmarca todas as opções no dropdown de Incremento
        for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
            if option.is_selected():
                option.click()
                #print("Opção desmarcada no dropdown de Incremento:", option.text)

        # Seleciona todas as opções no dropdown de Incremento novamente
        for option in dropdown_incremento.find_elements(By.TAG_NAME, 'option'):
            option.click()
            #print("Opção selecionada no dropdown de Incremento:", option.text)


        # Zera as opções do dropdown de Arquivos
        dropdown_arquivos = driver.find_element(By.XPATH, "//select[@id='A']")

        # Desmarca todas as opções selecionadas
        for option in dropdown_arquivos.find_elements(By.TAG_NAME, 'option'):
            if option.is_selected():
                option.click()

        # Seleciona todos os meses de 2019 no dropdown de Arquivos
        #ano = '2019'
        for option in dropdown_arquivos.find_elements(By.TAG_NAME, 'option'):
            if ano in option.text:
                option.click()
                #print("Opção selecionada no dropdown de Arquivos:", option.text)
                ano_selecionado = option.text

        # Marca a opção "Tabela com bordas"
        tabela_com_bordas = driver.find_element(By.XPATH, "//input[@value='table']")
        tabela_com_bordas.click()
        #print("Opção 'Tabela com bordas' marcada")

        # Clicar no botão "Mostra"
        botao_mostra = driver.find_element(By.XPATH, "//input[@class='mostra']")
        botao_mostra.click()
        #print("Botão 'Mostra' clicado")

        # Espera até que uma nova aba seja aberta
        WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))

        # Alterna para a nova aba
        janela_atual = driver.window_handles[1]
        driver.switch_to.window(janela_atual)

        # Clique no botão "Copia como .CSV"
        botao_csv = driver.find_element(By.XPATH, "//a[contains(text(), 'Copia como .CSV')]")
        botao_csv.click()
        #print("Botão 'Copia como .CSV' clicado")

        # Aguarda um tempo para o download ser concluído
        time.sleep(5)

        # Captura o nome do arquivo baixado
        arquivos_csv = glob.glob(os.path.join(download_path, "sih_cnv_*.csv"))  
        arquivo_baixado = arquivos_csv[0] if arquivos_csv else None

        if arquivo_baixado is not None:
            #print(f"Arquivo baixado: {os.path.basename(arquivo_baixado)}")
            #print(f"Localização: {os.path.dirname(arquivo_baixado)}")
            arquivo_novo = os.path.join(download_path, f"{uf}_{ano}.csv")
            os.rename(arquivo_baixado, arquivo_novo)
            #print(f"Arquivo '{arquivo_baixado}' renomeado para '{os.path.basename(arquivo_novo)}'")
            print(f"'{os.path.basename(arquivo_novo)}' baixado em csv_files.")

        else:
            print("Nenhum arquivo CSV foi encontrado.")

    finally:
        driver.quit()

#uf_desejada = 'pb'
#ano_desejado = '2019'
#get_data_por_ano(uf_desejada, ano_desejado)

## Loop por todas as UFs

In [24]:
anos = ['2019', '2020', '2021', '2022', '2023']
uf_desejada = ['ac', 'al', 'am', 'ap', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mg', 'ms', 'mt', 'pa', 'pb',
       'pe', 'pi', 'pr', 'rj', 'rn', 'ro', 'rr', 'rs', 'sc', 'se', 'sp', 'to']

for uf in uf_desejada:
    for ano in anos:
        get_data_por_ano(uf, ano)

Baixando CSV da UF ac do ano 2019...
'ac_2019.csv' baixado em csv_files.
Baixando CSV da UF ac do ano 2020...
'ac_2020.csv' baixado em csv_files.
Baixando CSV da UF ac do ano 2021...
'ac_2021.csv' baixado em csv_files.
Baixando CSV da UF ac do ano 2022...
'ac_2022.csv' baixado em csv_files.
Baixando CSV da UF ac do ano 2023...
'ac_2023.csv' baixado em csv_files.
Baixando CSV da UF al do ano 2019...
'al_2019.csv' baixado em csv_files.
Baixando CSV da UF al do ano 2020...
'al_2020.csv' baixado em csv_files.
Baixando CSV da UF al do ano 2021...
'al_2021.csv' baixado em csv_files.
Baixando CSV da UF al do ano 2022...
'al_2022.csv' baixado em csv_files.
Baixando CSV da UF al do ano 2023...
'al_2023.csv' baixado em csv_files.
Baixando CSV da UF am do ano 2019...
'am_2019.csv' baixado em csv_files.
Baixando CSV da UF am do ano 2020...
'am_2020.csv' baixado em csv_files.
Baixando CSV da UF am do ano 2021...
'am_2021.csv' baixado em csv_files.
Baixando CSV da UF am do ano 2022...
'am_2022.csv' 